In [1]:
import os


In [2]:
%pwd

'c:\\testproj2\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\testproj2'

In [4]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [7]:
data.shape

(1599, 12)

In [8]:
# Import necessary modules  
from dataclasses import dataclass  # Provides a decorator to create data classes  
from pathlib import Path  # Helps in handling file system paths  

# Define a data class for storing data validation configuration  
@dataclass  
class DataValidationConfig:  
    root_dir: Path  # The root directory where validation-related files are stored  
    STATUS_FILE: str  # Path to the status file that tracks validation results  
    unzip_data_dir: Path  # Directory where unzipped data is stored for validation  
    all_schema: dict  # Dictionary containing schema definitions for validation  


In [9]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [10]:
# Define the ConfigurationManager class to manage configuration settings
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Path to the main configuration YAML file
        params_filepath=PARAMS_FILE_PATH,  # Path to the parameters YAML file
        schema_filepath=SCHEMA_FILE_PATH,  # Path to the schema YAML file
    ):
        # Load configuration settings from the provided YAML files
        self.config = read_yaml(config_filepath)  # Reads main configuration file
        self.params = read_yaml(params_filepath)  # Reads parameters file
        self.schema = read_yaml(schema_filepath)  # Reads schema file

        # Ensure that the necessary directories exist by creating them if missing
        create_directories([self.config.artifacts_root])

    # Function to retrieve the Data Validation configuration
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation  # Access data validation settings
        schema = self.schema.COLUMNS  # Extract schema column definitions from YAML

        # Ensure the root directory for data validation exists
        create_directories([config.root_dir])

        # Create a DataValidationConfig instance with extracted settings
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,  # Directory for validation-related files
            STATUS_FILE=config.STATUS_FILE,  # Path to the validation status file
            unzip_data_dir=config.unzip_data_dir,  # Directory where unzipped data is stored
            all_schema=schema,  # Schema dictionary containing column definitions
        )

        return data_validation_config  # Return the configured DataValidationConfig object


In [11]:
import os
from src.datascience import logger

In [12]:
# Import necessary modules
import pandas as pd  # Used for handling and processing CSV files

# Define the DataValidation class to validate data against a predefined schema
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        """
        Initializes the DataValidation class with a configuration object.

        Parameters:
        config (DataValidationConfig): Configuration object containing validation settings.
        """
        self.config = config  # Store the configuration settings

    def validate_all_columns(self) -> bool:
        """
        Validates if all the required columns are present in the dataset.

        Returns:
        bool: True if validation is successful, False otherwise.
        """
        try:
            validation_status = None  # Initialize validation status as None

            # Read the CSV file containing the data to be validated
            data = pd.read_csv(self.config.unzip_data_dir)

            # Get the list of all columns present in the dataset
            all_cols = list(data.columns)

            # Retrieve the expected schema columns from the configuration
            all_schema = self.config.all_schema.keys()

            # Iterate over each column in the dataset
            for col in all_cols:
                # Check if the column exists in the predefined schema
                if col not in all_schema:
                    validation_status = False  # Mark validation as failed
                    with open(self.config.STATUS_FILE, 'w') as f:  # Open status file for writing
                        f.write(f"Validation status: {validation_status}")  # Log the status
                else:
                    validation_status = True  # Mark validation as successful
                    with open(self.config.STATUS_FILE, 'w') as f:  # Open status file for writing
                        f.write(f"Validation status: {validation_status}")  # Log the status

            return validation_status  # Return final validation result

        except Exception as e:
            raise e  # Raise any exceptions encountered during validation


In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-02-06 22:22:10,424: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-06 22:22:10,426: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 22:22:10,429: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 22:22:10,429: INFO: common: created directory at: artifacts]
[2025-02-06 22:22:10,431: INFO: common: created directory at: artifacts/data_validation]
